# 🚗 Parking Slot Detection - YOLOv8 Training

This notebook trains a YOLOv8 Nano model to detect:
- **Cars** (occupied parking slots)
- **Empty slots** (available parking spaces)

## Instructions
1. **Runtime > Change runtime type > GPU (T4)**
2. Upload your `formatted_dataset.zip` when prompted
3. Run all cells sequentially
4. Download the trained model at the end

## 1. Setup Environment

In [ ]:
# Install required packages
!pip install ultralytics -q

# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

## 2. Upload Your Dataset

**Before running this cell:**
1. On your local machine, zip the `formatted_dataset` folder:
   - Navigate to `C:\Users\akshaya\Desktop\mini`
   - Right-click `formatted_dataset` → Send to → Compressed (zipped) folder
   - This creates `formatted_dataset.zip`
2. Run the cell below and upload the zip file when prompted

In [ ]:
from google.colab import files
import zipfile
import os

# Upload the dataset zip file
print("📁 Please upload 'formatted_dataset.zip'...")
uploaded = files.upload()

# Extract the dataset
zip_filename = list(uploaded.keys())[0]
print(f"\n📦 Extracting {zip_filename}...")

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('/content/')

print("✅ Dataset extracted successfully!")

# List contents
!ls -la /content/formatted_dataset/

## 3. Create Dataset Configuration (data.yaml)

In [ ]:
# Create data.yaml configuration file
data_yaml_content = """path: /content/formatted_dataset
train: images/train
val: images/val

names:
  0: car
  1: empty
"""

with open('/content/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("✅ data.yaml created!")
print("\n📄 Contents:")
!cat /content/data.yaml

## 4. Verify Dataset Structure

In [ ]:
import os

# Check dataset structure
base_path = '/content/formatted_dataset'

train_images = os.path.join(base_path, 'images', 'train')
train_labels = os.path.join(base_path, 'labels', 'train')
val_images = os.path.join(base_path, 'images', 'val')
val_labels = os.path.join(base_path, 'labels', 'val')

print("📊 Dataset Statistics:")
print(f"  Train Images: {len(os.listdir(train_images)) if os.path.exists(train_images) else 'NOT FOUND'}")
print(f"  Train Labels: {len(os.listdir(train_labels)) if os.path.exists(train_labels) else 'NOT FOUND'}")
print(f"  Val Images:   {len(os.listdir(val_images)) if os.path.exists(val_images) else 'NOT FOUND'}")
print(f"  Val Labels:   {len(os.listdir(val_labels)) if os.path.exists(val_labels) else 'NOT FOUND'}")

# Preview a few labels
print("\n🔍 Sample Labels (first 3):")
label_files = os.listdir(train_labels)[:3]
for lf in label_files:
    print(f"\n--- {lf} ---")
    with open(os.path.join(train_labels, lf), 'r') as f:
        content = f.read()
        print(content if content else "(empty file)")

## 5. Train YOLOv8 Model 🚀

This cell trains the model. Training time depends on:
- Dataset size
- Number of epochs
- GPU type

**Estimated time: 10-30 minutes** for 50 epochs with ~2000 images on T4 GPU.

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 Nano model
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data='/content/data.yaml',
    epochs=50,           # Adjust as needed (more epochs = better accuracy but longer time)
    imgsz=640,           # Image size
    batch=16,            # Batch size (reduce if you get OOM errors)
    device=0,            # Use GPU
    workers=2,           # Number of data loading workers
    patience=10,         # Early stopping patience
    plots=True,          # Generate training plots
    save=True,           # Save checkpoints
    project='/content/runs/detect',
    name='parking_detector'
)

print("\n✅ Training Complete!")
print(f"📁 Model saved at: {results.save_dir}")

## 6. Validate the Model

In [ ]:
# Load the best trained model
best_model = YOLO('/content/runs/detect/parking_detector/weights/best.pt')

# Run validation
metrics = best_model.val()

print("\n📈 Validation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")

## 7. Visualize Training Results

In [ ]:
from IPython.display import Image, display
import os

results_dir = '/content/runs/detect/parking_detector/'

# Display training plots
plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'P_curve.png', 'R_curve.png']

for plot in plots:
    plot_path = os.path.join(results_dir, plot)
    if os.path.exists(plot_path):
        print(f"\n📊 {plot}:")
        display(Image(filename=plot_path, width=800))

## 8. Test on Sample Images

In [ ]:
from IPython.display import Image, display
import os

# Run inference on validation images
val_images_dir = '/content/formatted_dataset/images/val'
sample_images = os.listdir(val_images_dir)[:5]  # First 5 images

print("🔍 Running inference on sample images...\n")

for img_name in sample_images:
    img_path = os.path.join(val_images_dir, img_name)
    results = best_model(img_path)
    
    # Save result
    result_path = f'/content/result_{img_name}'
    results[0].save(result_path)
    
    print(f"📸 {img_name}:")
    display(Image(filename=result_path, width=600))
    print()

## 9. Download Trained Model 📥

Download the trained model to use in your local Streamlit app.

**After downloading:**
1. Place `best.pt` in `C:\Users\akshaya\Desktop\mini\`
2. Run `streamlit run app.py`
3. Select `best.pt` from the model dropdown in the sidebar

In [ ]:
from google.colab import files
import shutil

# Path to the best model
best_model_path = '/content/runs/detect/parking_detector/weights/best.pt'
last_model_path = '/content/runs/detect/parking_detector/weights/last.pt'

# Copy to a simpler name
shutil.copy(best_model_path, '/content/best.pt')

print("📦 Downloading best.pt...")
print("\n💡 After download, place this file in your 'mini' project folder!")

# Download the model
files.download('/content/best.pt')

## 10. (Optional) Download All Training Artifacts

In [ ]:
from google.colab import files
import shutil

# Zip all training results
shutil.make_archive('/content/training_results', 'zip', '/content/runs/detect/parking_detector')

print("📦 Downloading all training artifacts...")
files.download('/content/training_results.zip')

---

## 🎉 Done!

You have successfully trained a parking slot detection model!

### Next Steps:
1. Download `best.pt` to your local machine
2. Place it in `C:\Users\akshaya\Desktop\mini\`
3. Run `streamlit run app.py`
4. Select `best.pt` from the model dropdown
5. Upload an image or video to test!

### Model Classes:
- **Class 0: car** - Detected vehicles (occupied slots)
- **Class 1: empty** - Empty parking slots (available)